In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
movies = pd.DataFrame({
    'movie_id': [1, 2, 3, 4, 5],
    'title': ["Inception", "Avengers", "Titanic", "The Matrix", "Toy Story"],
    'genres': ["Action Sci-Fi", "Action Adventure", "Romance Drama", "Action Sci-Fi", "Animation Comedy"]
})

# User-item rating matrix
ratings = pd.DataFrame({
    'user_id': [1, 1, 1, 2, 2, 3, 3, 4],
    'movie_id': [1, 2, 3, 1, 4, 2, 5, 5],
    'rating': [5, 4, 3, 5, 4, 3, 4, 5]
})

In [9]:
user_movie_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

In [10]:
user_similarity = cosine_similarity(user_movie_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

In [11]:
def recommend_collaborative(user_id, num_recommendations=3):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]
    similar_user_ratings = user_movie_matrix.loc[similar_users]
    
    # Calculate weighted average ratings based on similar users
    user_ratings = similar_user_ratings.mean(axis=0)
    user_ratings = user_ratings[user_movie_matrix.loc[user_id] == 0]  # Exclude already rated movies
    recommended_movies = user_ratings.sort_values(ascending=False).head(num_recommendations)
    
    return movies[movies['movie_id'].isin(recommended_movies.index)]['title'].tolist()

In [12]:
print("Collaborative Filtering Recommendations for User 1:", recommend_collaborative(user_id=1))

Collaborative Filtering Recommendations for User 1: ['The Matrix', 'Toy Story']


In [13]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genres'])

In [14]:
movie_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
movie_similarity_df = pd.DataFrame(movie_similarity, index=movies['movie_id'], columns=movies['movie_id'])

In [15]:
def recommend_content_based(movie_id, num_recommendations=3):
    similar_movies = movie_similarity_df[movie_id].sort_values(ascending=False).index[1:num_recommendations+1]
    return movies[movies['movie_id'].isin(similar_movies)]['title'].tolist()

In [16]:
print("Content-Based Recommendations for 'Inception':", recommend_content_based(movie_id=1))

Content-Based Recommendations for 'Inception': ['Avengers', 'Titanic', 'The Matrix']
